**Signal Source seperation (de-noising)** [on regression principle]

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Softmax,LSTM,GRU
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import librosa
import math
from IPython.display import Audio
import os
import pickle

In [ ]:
# a=np.array([[1,2,3],[4,5,6],[7,8,9]])
# np.save('temp.npy',a)
# a1=np.load('temp.npy')
# a1

In [ ]:
# getting the filenames
res=[]
for root, dirs, files in os.walk("/content/drive/My Drive/Deep_Learning/HW3/timit-homework/tr/"):
  for filename in files:
    res.append(filename)
trx, trs, trn = [], [], []
for filename in res:
  if filename[:3] == "trx":
    trx.append(filename)
  elif filename[:3] == "trs":
    trs.append(filename)
  elif filename[:3] == "trn":
    trn.append(filename)

trs.sort()
trx.sort()
trn.sort()

In [ ]:
directory= "/content/drive/My Drive/Deep_Learning/HW3/timit-homework/tr/"

In [ ]:
## Loading the wav files using librsoa
TRX,TRS,TRN=[],[],[]
#s,sr=librosa.load(path=directory+trs[0])
for filename in trx:
  TRX.append(librosa.load(path=directory+filename,sr=None)[0])
for filename in trs:
  TRS.append(librosa.load(path=directory+filename,sr=None)[0])

for filename in trn:
  TRN.append(librosa.load(path=directory+filename,sr=None)[0])


In [ ]:
# calculating the stft
Xtr,Str,Ntr=[],[],[]
Xtr_abs,Str_abs,Ntr_abs=[],[],[]
for x in TRX:
  Xtr.append(librosa.stft(x, n_fft=1024, hop_length=512))

for x in TRS:
  Str.append(librosa.stft(x, n_fft=1024, hop_length=512))

for x in TRN:
  Ntr.append(librosa.stft(x, n_fft=1024, hop_length=512))

#getting the magnitude spectra
for x in Xtr:
  Xtr_abs.append(np.abs(x))

for x in Str:
  Str_abs.append(np.abs(x))

for x in Ntr:
  Ntr_abs.append(np.abs(x))


In [ ]:
## saving the lists in pickle objects
file_name = "Xtr.pkl"
open_file = open(directory+file_name, "wb")
pickle.dump(Xtr, open_file)
open_file.close()

file_name = "Str.pkl"
open_file = open(directory+file_name, "wb")
pickle.dump(Str, open_file)
open_file.close()

file_name = "Ntr.pkl"
open_file = open(directory+file_name, "wb")
pickle.dump(Ntr, open_file)
open_file.close()

file_name = "Xtr_abs.pkl"
open_file = open(directory+file_name, "wb")
pickle.dump(Xtr_abs, open_file)
open_file.close()

file_name = "Str_abs.pkl"
open_file = open(directory+file_name, "wb")
pickle.dump(Str_abs, open_file)
open_file.close()

file_name = "Ntr_abs.pkl"
open_file = open(directory+file_name, "wb")
pickle.dump(Ntr_abs, open_file)
open_file.close()

In [ ]:
## Loading the spectrum lists from pickle
file_name= "Xtr_abs.pkl"
open_file = open(directory+file_name, "rb")
Xtr_abs = pickle.load(open_file)
open_file.close()

file_name= "Str_abs.pkl"
open_file = open(directory+file_name, "rb")
Str_abs = pickle.load(open_file)
open_file.close()

file_name= "Ntr_abs.pkl"
open_file = open(directory+file_name, "rb")
Ntr_abs = pickle.load(open_file)
open_file.close()

file_name= "Xtr.pkl"
open_file = open(directory+file_name, "rb")
Xtr = pickle.load(open_file)
open_file.close()

file_name= "Str.pkl"
open_file = open(directory+file_name, "rb")
Str = pickle.load(open_file)
open_file.close()

file_name= "Ntr.pkl"
open_file = open(directory+file_name, "rb")
Ntr = pickle.load(open_file)
open_file.close()


In [ ]:
# trucating the samples
temp=[arr.shape[1] for arr in Xtr_abs]
Xtr_abs_trunc,Str_abs_trunc,Ntr_abs_trunc,Xtr_trunc,Str_trunc,Ntr_trunc=[],[],[],[],[],[]
for arr in Xtr_abs:
  Xtr_abs_trunc.append(arr[:,:min(temp)])

for arr in Str_abs:
  Str_abs_trunc.append(arr[:,:min(temp)])

for arr in Ntr_abs:
  Ntr_abs_trunc.append(arr[:,:min(temp)])

for arr in Xtr:
  Xtr_trunc.append(arr[:,:min(temp)])

for arr in Str:
  Str_trunc.append(arr[:,:min(temp)])

for arr in Ntr:
  Ntr_trunc.append(arr[:,:min(temp)])


In [ ]:
# reshaping the lists of arrays into nd arrdays
Xtr_abs=np.array(Xtr_abs_trunc).reshape((1200,513,min(temp)))
Str_abs=np.array(Str_abs_trunc).reshape((1200,513,min(temp)))
Ntr_abs=np.array(Ntr_abs_trunc).reshape((1200,513,min(temp)))

Xtr=np.array(Xtr_trunc).reshape((1200,513,min(temp)))
Str=np.array(Str_trunc).reshape((1200,513,min(temp)))
Ntr=np.array(Ntr_trunc).reshape((1200,513,min(temp)))
print("shapes",Xtr_abs.shape,Str_abs.shape,Ntr_abs.shape,Xtr.shape,Str.shape,Ntr.shape)

shapes (1200, 513, 39) (1200, 513, 39) (1200, 513, 39) (1200, 513, 39) (1200, 513, 39) (1200, 513, 39)


Similarly loading the validation set

In [ ]:
directory_val= "/content/drive/My Drive/Deep_Learning/HW3/timit-homework/v/"

In [ ]:
# getting the filenames
res=[]
for root, dirs, files in os.walk(directory_val):
  for filename in files:
    res.append(filename)
vx, vn, vs = [], [], []
for filename in res:
  if filename[:2] == "vx":
    vx.append(filename)
  elif filename[:2] == "vs":
    vs.append(filename)
  elif filename[:2] == "vn":
    vn.append(filename)

vs.sort()
vx.sort()
vn.sort()

## Loading the wav files using librsoa
VX,VS,VN=[],[],[]
#s,sr=librosa.load(path=directory+vs[0])
for filename in vx:
  VX.append(librosa.load(path=directory_val+filename,sr=None)[0])
for filename in vs:
  VS.append(librosa.load(path=directory_val+filename,sr=None)[0])
for filename in vn:
  VN.append(librosa.load(path=directory_val+filename,sr=None)[0])

# calculating the stft
Xval,Sval,Nval=[],[],[]
Xval_abs,Sval_abs,Nval_abs=[],[],[]
for x in VX:
  Xval.append(librosa.stft(x, n_fft=1024, hop_length=512))

for x in VS:
  Sval.append(librosa.stft(x, n_fft=1024, hop_length=512))

for x in VN:
  Nval.append(librosa.stft(x, n_fft=1024, hop_length=512))

#getting the magnitude spectra
for x in Xval:
  Xval_abs.append(np.abs(x))

for x in Sval:
  Sval_abs.append(np.abs(x))

for x in Nval:
  Nval_abs.append(np.abs(x))


## saving the lists in pickle objects
file_name = "Xval.pkl"
open_file = open(directory_val+file_name, "wb")
pickle.dump(Xval, open_file)
open_file.close()

file_name = "Sval.pkl"
open_file = open(directory_val+file_name, "wb")
pickle.dump(Sval, open_file)
open_file.close()

file_name = "Nval.pkl"
open_file = open(directory_val+file_name, "wb")
pickle.dump(Nval, open_file)
open_file.close()

file_name = "Xval_abs.pkl"
open_file = open(directory_val+file_name, "wb")
pickle.dump(Xval_abs, open_file)
open_file.close()

file_name = "Sval_abs.pkl"
open_file = open(directory_val+file_name, "wb")
pickle.dump(Sval_abs, open_file)
open_file.close()

file_name = "Nval_abs.pkl"
open_file = open(directory_val+file_name, "wb")
pickle.dump(Nval_abs, open_file)
open_file.close()

print("-----------Saving Done-----------------")

In [ ]:
## Loading the spectrum lists from pickle

file_name= "Xval_abs.pkl"
open_file = open(directory_val+file_name, "rb")
Xval_abs = pickle.load(open_file)
open_file.close()

file_name= "Xval.pkl"
open_file = open(directory_val+file_name, "rb")
Xval = pickle.load(open_file)
open_file.close()

file_name= "Sval_abs.pkl"
open_file = open(directory_val+file_name, "rb")
Sval_abs = pickle.load(open_file)
open_file.close()

file_name= "Sval.pkl"
open_file = open(directory_val+file_name, "rb")
Sval = pickle.load(open_file)
open_file.close()

file_name= "Nval_abs.pkl"
open_file = open(directory_val+file_name, "rb")
Nval_abs = pickle.load(open_file)
open_file.close()

file_name= "Nval.pkl"
open_file = open(directory_val+file_name, "rb")
Nval = pickle.load(open_file)
open_file.close()


In [ ]:
def truncate(Xtr,Str,Ntr,Xtr_abs,Str_abs,Ntr_abs):
# trucating the samples
  temp=[arr.shape[1] for arr in Xtr_abs]
  Xtr_abs_trunc,Str_abs_trunc,Ntr_abs_trunc,Xtr_trunc,Str_trunc,Ntr_trunc=[],[],[],[],[],[]
  for arr in Xtr_abs:
    Xtr_abs_trunc.append(arr[:,:min(temp)])

  for arr in Str_abs:
    Str_abs_trunc.append(arr[:,:min(temp)])

  for arr in Ntr_abs:
    Ntr_abs_trunc.append(arr[:,:min(temp)])

  for arr in Xtr:
    Xtr_trunc.append(arr[:,:min(temp)])

  for arr in Str:
    Str_trunc.append(arr[:,:min(temp)])

  for arr in Ntr:
    Ntr_trunc.append(arr[:,:min(temp)])

  return Xtr_abs_trunc,Str_abs_trunc,Ntr_abs_trunc,Xtr_trunc,Str_trunc,Ntr_trunc

In [ ]:
Xval_abs_trunc,Sval_abs_trunc,Nval_abs_trunc,Xval_trunc,Sval_trunc,Nval_trunc = truncate(Xval,Sval,Nval,Xval_abs,Sval_abs,Nval_abs)

In [ ]:
def reshape(Xtr_abs_trunc,Str_abs_trunc,Ntr_abs_trunc,Xtr_trunc,Str_trunc,Ntr_trunc):
# reshaping the lists of arrays into nd arrdays
  temp=[39,59]
  Xtr_abs=np.array(Xtr_abs_trunc).reshape((1200,513,min(temp)))
  Str_abs=np.array(Str_abs_trunc).reshape((1200,513,min(temp)))
  Ntr_abs=np.array(Ntr_abs_trunc).reshape((1200,513,min(temp)))

  Xtr=np.array(Xtr_trunc).reshape((1200,513,min(temp)))
  Str=np.array(Str_trunc).reshape((1200,513,min(temp)))
  Ntr=np.array(Ntr_trunc).reshape((1200,513,min(temp)))
  print("shapes",Xtr_abs.shape,Str_abs.shape,Ntr_abs.shape,Xtr.shape,Str.shape,Ntr.shape)

  return Xtr,Str,Ntr,Xtr_abs,Str_abs,Ntr_abs

Xval,Sval,Nval,Xval_abs,Sval_abs,Nval_abs= reshape(Xval_abs_trunc,Sval_abs_trunc,Nval_abs_trunc,Xval_trunc,Sval_trunc,Nval_trunc)

shapes (1200, 513, 39) (1200, 513, 39) (1200, 513, 39) (1200, 513, 39) (1200, 513, 39) (1200, 513, 39)


In [ ]:
Xval_abs.shape,Sval_abs.shape,Nval_abs.shape,Xval.shape,Sval.shape,Nval.shape

((1200, 513, 39),
 (1200, 513, 39),
 (1200, 513, 39),
 (1200, 513, 39),
 (1200, 513, 39),
 (1200, 513, 39))

Constructing training and validation labels and inputs

In [ ]:
#constructing the target mask for training and validaion
M= np.where(Str_abs>Ntr_abs,1,0)
print(M.shape)

M_val= np.where(Sval_abs>Nval_abs,1,0)
print(M_val.shape)

(1200, 513, 39)
(1200, 513, 39)


In [ ]:
## constructiong the training and validation data set

X_fit=Xtr_abs.reshape((1200,39,513))
Y_fit=M.reshape((1200,39,513))
print("training shapes",X_fit.shape,Y_fit.shape)

X_validation= Xval_abs.reshape((1200,39,513))
Y_validation=M_val.reshape((1200,39,513))

print("validation shapes",X_validation.shape,Y_validation.shape)

training shapes (1200, 39, 513) (1200, 39, 513)
validation shapes (1200, 39, 513) (1200, 39, 513)


In [ ]:
## constructing the LSTM network
def weights():
  initializer = tf.keras.initializers.HeNormal
  return initializer

model=Sequential()
model.add(LSTM(513, input_shape=(39,513),kernel_initializer=weights(), return_sequences=True))
model.add(keras.layers.Dropout(0.005))

model.add(LSTM(1024,kernel_initializer=weights(),return_sequences=True))
model.add(keras.layers.Dropout(0.005))

#model.add(LSTM(1024,activation='relu',kernel_initializer=weights(),return_sequences=True))
##output layer
model.add(Dense(513, activation='relu'))
#compiling the model
opt= tf.keras.optimizers.Adam()
mse=tf.keras.losses.MeanSquaredError()
model.compile(optimizer=opt,loss=mse,metrics=['accuracy'],)
# training the model
history=model.fit(X_fit,Y_fit, validation_data=(X_validation,Y_validation),epochs=80,verbose=2,batch_size=50)

Epoch 1/80
24/24 - 2s - loss: 0.2804 - accuracy: 3.8462e-04 - val_loss: 0.2391 - val_accuracy: 5.9829e-04
Epoch 2/80
24/24 - 2s - loss: 0.2372 - accuracy: 0.0010 - val_loss: 0.2252 - val_accuracy: 4.2735e-04
Epoch 3/80
24/24 - 2s - loss: 0.2302 - accuracy: 5.5556e-04 - val_loss: 0.2344 - val_accuracy: 4.9145e-04
Epoch 4/80
24/24 - 2s - loss: 0.2288 - accuracy: 0.0015 - val_loss: 0.2199 - val_accuracy: 0.0024
Epoch 5/80
24/24 - 2s - loss: 0.2214 - accuracy: 0.0026 - val_loss: 0.2139 - val_accuracy: 0.0014
Epoch 6/80
24/24 - 2s - loss: 0.2181 - accuracy: 8.9744e-04 - val_loss: 0.2198 - val_accuracy: 0.0012
Epoch 7/80
24/24 - 2s - loss: 0.2130 - accuracy: 0.0011 - val_loss: 0.2112 - val_accuracy: 0.0018
Epoch 8/80
24/24 - 2s - loss: 0.2108 - accuracy: 0.0022 - val_loss: 0.2100 - val_accuracy: 0.0027
Epoch 9/80
24/24 - 2s - loss: 0.2093 - accuracy: 0.0020 - val_loss: 0.2080 - val_accuracy: 0.0016
Epoch 10/80
24/24 - 2s - loss: 0.2048 - accuracy: 0.0015 - val_loss: 0.2036 - val_accuracy: 9.

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 39, 513)           2107404   
_________________________________________________________________
dropout_2 (Dropout)          (None, 39, 513)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 39, 1024)          6299648   
_________________________________________________________________
dropout_3 (Dropout)          (None, 39, 1024)          0         
_________________________________________________________________
dense_1 (Dense)              (None, 39, 513)           525825    
Total params: 8,932,877
Trainable params: 8,932,877
Non-trainable params: 0
_________________________________________________________________


Unmasking the validation predictions

In [ ]:
Mv_pred=model.predict(X_validation)
## getting back original shapes
Mv_pred= Mv_pred.reshape((1200,513,39))

Sv_pred= Mv_pred*Xval # *Xval_abs
# ## ISTFT on frequency domain validation signal
sv_preds=[]
for i in range(1200):
  sv_preds.append(librosa.istft(Sv_pred[i,:,:], hop_length=512))

SNR Calculation

In [ ]:
sv=[]
for i in range(1200):
  sv.append(librosa.istft(Sval[i,:,:], hop_length=512))
print(len(sv))

1200


In [ ]:
## calculating the SNR on validation set
snr=[]
for i in range(1200):
  snr.append(10*np.log10(np.sum(sv[i]**2)/np.sum((sv[i]-sv_preds[i])**2)))

In [ ]:
print("Max SNR value", max(snr))
print("Min SNR value", min(snr))
print("Average SNR value", np.mean(snr))

Max SNR value 20.609419345855713
Min SNR value -0.24631641805171967
Average SNR value 9.73469845596701


In [ ]:
# listening to a random signal (6th audio)
librosa.output.write_wav('validation1.wav', sv_preds[5],16000)
Audio('validation1.wav')

Test Data Set

In [ ]:
## reading the test files
directory_test= "/content/drive/My Drive/Deep_Learning/HW3/timit-homework/te/"

In [ ]:

# getting the filenames
res=[]
for root, dirs, files in os.walk(directory_test):
  for filename in files:
    res.append(filename)
res.sort()

TE=[]

for filename in res:
  TE.append(librosa.load(path=directory_test+filename,sr=None)[0])
  sr=librosa.load(path=directory_test+filename,sr=None)[1]

#getting magnitude spectra of all test files
specs_te=[]
specs_te_complex=[]
for x in TE:
  specs_te.append(np.abs(librosa.stft(x, n_fft=1024, hop_length=512)))
  specs_te_complex.append(librosa.stft(x, n_fft=1024, hop_length=512))

In [ ]:
## saving the lists in pickle objects
file_name = "specs_te.pkl"
open_file = open(directory_test+file_name, "wb")
pickle.dump(specs_te, open_file)
open_file.close()

In [ ]:
## Loading the spectrum lists from pickle
file_name = "specs_te.pkl"
open_file = open(directory_test+file_name, "rb")
specs_te = pickle.load(open_file)
open_file.close()


In [ ]:
temp=[arr.shape[1] for arr in specs_te]
specs_te_trunc=[]
for arr in specs_te:
  specs_te_trunc.append(arr[:,:min(temp)])

In [ ]:
temp=[arr.shape[1] for arr in specs_te_complex]
specs_te_trunc_complex=[]
for arr in specs_te_complex:
  specs_te_trunc_complex.append(arr[:,:min(temp)])

In [ ]:
specs_te=np.array(specs_te_trunc).reshape((400,513,min(temp)))
print(specs_te.shape)

(400, 513, 56)


In [ ]:
specs_te_complex=np.array(specs_te_trunc_complex).reshape((400,513,min(temp)))
print(specs_te_complex.shape)

(400, 513, 56)


In [ ]:
## prediction on the test signals
test_preds=model.predict(specs_te.reshape((400,56,513)))

In [ ]:
test_preds.shape

(400, 56, 513)

In [ ]:
##istft on test preds
test_preds=test_preds.reshape((400,513,56))
test_preds=test_preds*specs_te_complex
st=[]
for i in range(400):
  st.append(librosa.istft(test_preds[i,:,:], hop_length=512))
print(len(st))

400


In [ ]:
output_directory= "/content/drive/My Drive/Deep_Learning/HW3/timit-homework/output/"

In [ ]:
##writing into .wav files
for i in range(400):
  filename=f'signal{i}'
  librosa.output.write_wav(output_directory+filename, st[i],16000)